<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/QR_with_title.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyqrcode pypng

In [ ]:
import pyqrcode
from IPython.display import Image, display, HTML
from pyqrcode import QRCode

s = input("Paste the link: ")
t = input("Title of the QR: ")
# Generate QR code
url = pyqrcode.create(s)

# Create and save the png file naming "myqr.png"
url.png("myqr.png", scale=10)

def show_png_with_title(file, title="QR Code"):
    # Display the title as HTML to enhance styling options (like centering)
    display(HTML(f"<h1 style='text-align: center;'>{title}</h1>"))
    # Display the QR code image
    display(Image(file))

# Call the function with the title you want above the QR code
show_png_with_title("myqr.png", title=t)


In [ ]:
#@markdown QR code with title (on Colab)
import pyqrcode
import base64
from IPython.display import Image, display, HTML

s = input("Paste the link: ")

# Generate QR code
url = pyqrcode.create(s)

# Create and save the png file naming "myqr.png"
url.png("myqr.png", scale=10)

def show_png_with_title(file, title="QR Code"):
    # Encode image to base64
    with open(file, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

    # Display the title and the image as HTML with improved styling for alignment
    html = f"""
    <div style="text-align: center; margin: 10px 0;">
        <h1>{title}</h1>
        <img src="data:image/png;base64,{encoded_string}" style="margin-top: 10px;"/>
    </div>
    """
    display(HTML(html))

# Call the function with the title you want above the QR code
show_png_with_title("myqr.png", title="Scan this QR Code")


# Gradio implementation (This is deployed on Huggingface)

+ This requires ttf font file to upload: [font link](https://github.com/owncloud/docs/blob/master/fonts/dejavu-sans-bold.ttf)
+  [App link](https://mk-316-QRcode-with-title.hf.space)

In [ ]:
%%capture
!pip install gradio

In [ ]:
import gradio as gr
import pyqrcode
from PIL import Image, ImageDraw, ImageFont
import io

def generate_qr_code(url, title):
    # Generate QR code
    qr = pyqrcode.create(url)
    buffer = io.BytesIO()

    # Create the QR code as a PNG
    qr.png(buffer, scale=10)

    # Move to the beginning of the StringIO buffer
    buffer.seek(0)
    qr_image = Image.open(buffer)

    # Prepare to add title to the image
    image_width = qr_image.width
    image_height = qr_image.height + 70  # Add 70 pixels space for title

    # Create a new image with white background
    result_image = Image.new('RGB', (image_width, image_height), 'white')
    # Paste the QR code onto this new image
    result_image.paste(qr_image, (0, 70))

    # Draw the title on the image
    draw = ImageDraw.Draw(result_image)
    font_path = "dejavu-sans-bold.ttf"  # Ensure the font path is correct
    font = ImageFont.truetype(font_path, 24)  # Use a specific font size
    text_width, text_height = draw.textsize(title, font=font)
    text_x = (image_width - text_width) / 2  # Center the text
    draw.text((text_x, 20), title, fill="black", font=font)

    # Return the final image
    return result_image

# Create the Gradio interface
iface = gr.Interface(
    fn=generate_qr_code,
    inputs=[
        gr.Textbox(label="Enter URL", placeholder="Type or paste URL here..."),
        gr.Textbox(label="Enter Title for QR Code", placeholder="Type the title here...")
    ],
    outputs=gr.Image(label="QR Code Image", type="pil", format="png"),
    title="QR Code Generator",
    description="Enter a URL and a title to generate a QR Code. The title and the QR Code will be displayed in the same image."
)

iface.launch()
